In [1]:
import pandas as pd
import re
import mechanize
from bs4 import BeautifulSoup

In [ ]:
def search_company(date):
    # Date format: "2023-03-06"
    br = mechanize.Browser()
    br.set_handle_robots(False)
    br.set_handle_equiv(True)
    br.set_handle_gzip(True)
    br.set_handle_refresh(False)
    br.set_handle_redirect(True)
    br.set_handle_referer(True)

    headers = [
                (
                    "User-Agent",
                    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15",
                ),
                (   "Accept-Language", "en-GB,en;q=0.9"   ),
                (   "Accept-Encoding", "gzip, deflate, br"    ),
                (
                    "Accept",
                    "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                ),
                (   "Connection", "keep-alive"    ),
            ]

    br.addheaders = headers
    br.open("https://neu.insolvenzbekanntmachungen.de/ap/suche.jsf", timeout=5)
    #for form in br.forms():
    #    print("Form name:", form.name)
    #    print(form)
    br.select_form(name="frm_suche")
    # Browser passes through unknown attributes (including methods)
    # to the selected HTMLForm.
    br["frm_suche:ldi_datumVon:datumHtml5"] = date  # (the method here is __setitem__)
    br['frm_suche:ldi_datumBis:datumHtml5']= date
    # Submit current form.  Browser calls .close() on the current response on
    # navigation, so this closes response1
    response2 = br.submit()
    html = response2.read().decode("utf-8")
    result = pd.read_html(html, attrs={"id":"tbl_ergebnis"})[0]
    return result

In [ ]:
days = ["%.2d" % i for i in range(1,32)]
months = ["%.2d" % i for i in range(1,13)]
#months = ["%.2d" % i for i in range(1,4)] till March

In [ ]:
year = '2023'
all_2023 = pd.DataFrame()
for month in months:
    for day in days:
        date = year + "-" + month + "-" + day
        print(date)
        try:
            bs = search_company(date)
            all_2023 = pd.concat([all_2023,bs], axis=0)
        except:
            None

In [ ]:
all_2023.to_csv(r'insolvenzen_2023.csv',index=True, header=True)

In [ ]:
all_2020_23 = pd.concat([all_2021,all_2022,all_2023], axis=0)

In [ ]:
all_2020_23.to_csv(r'insolvenzen.csv',index=True, header=True)

In [2]:
all_2020_23 = pd.read_csv("insolvenzen.csv")

In [ ]:
companies_2020 = pd.read_csv("GER_dataframe.csv")

In [ ]:
companies_2021 = pd.read_csv("GER_dataframe_2021.csv")

In [5]:
all_2020_23['Name, Vorname / Bezeichnung']

0         Golfpark Schlossgut Sickendorf GmbH
1                              VIBROPLAN GmbH
2             "Camion Blue" Modevertrieb GmbH
3                      A-Z2 Bau GmbH & Co. KG
4                               Abdal, Sfiyan
                         ...                 
347498                            Zorn, Katja
347499                            Zorn, Katja
347500                           Zsolt, Szücs
347501                     Zühlsdorff, Annett
347502                     Zühlsdorff, Annett
Name: Name, Vorname / Bezeichnung, Length: 347503, dtype: object

In [15]:
companies_2021['Name des Beihilfeempfängers']

0                       SCHUNK GMBH & CO.KG SPANN- UND
1                                Flughafen Bremen GmbH
2                         Bernheimer Contemporary GmbH
3                                 Schlote Holding GmbH
4                                           ORSAY GmbH
                             ...                      
71166                                 Kulturwald gGmbH
71167                                       Kai Aßmann
71168                                    KultTour GmbH
71169                GOP Varieté München GmbH & Co. KG
71170    Spectrum Gastronomie- und Veranstaltungs GmbH
Name: Name des Beihilfeempfängers, Length: 71171, dtype: object

In [19]:
companies_2020['Name des Beihilfeempfängers'].isin(all_2020_23['Name, Vorname / Bezeichnung']).value_counts()

False    43872
True        71
Name: Name des Beihilfeempfängers, dtype: int64

In [16]:
companies_2021['Name des Beihilfeempfängers'].isin(all_2020_23['Name, Vorname / Bezeichnung']).value_counts()


False    70772
True       399
Name: Name des Beihilfeempfängers, dtype: int64

In [20]:
match_2022 = companies_2020['Name des Beihilfeempfängers'].isin(all_2020_23['Name, Vorname / Bezeichnung'])#.value_counts()

In [21]:
companies_2020['match'] = match_2022

In [17]:
match_2021 = companies_2021['Name des Beihilfeempfängers'].isin(all_2020_23['Name, Vorname / Bezeichnung'])#.value_counts()
companies_2021['match'] = match_2021